In [1]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 41.3 MB/s eta 0:00:00


In [67]:
from Bio import Entrez
from Bio import SeqIO
import time
import pandas as pd
import numpy as np

#Задание 1
Используя базу данных NCBI Protein, узнайть, есть ли у белка, который кодирует ген, известная 3D-структура. Добавьте в таблицу переменную Yes/No, в зависимости от результатов поиска.

In [68]:
genes_table= pd.read_csv('genes_table.csv')
genes_table.head()

,geneid,gene_symbol,gene_desc,locus,chromosome,chromosome_desc,seq_id,seq_start,seq_end,strand,seq,gene_syn,protein,q_transcripts,len_long_prot,seq_dna,snps_id,q_snps,Has 3D structure,Protein id with 3D
0,7157,TP53,tumor protein p53,17p13.1,NC_000017,"NC_000017.11 Homo sapiens chromosome 17, GRCh3...",568815581,7668421,7687489,minus,GGCAGCAAAGTTTTATTGTAAAATAAGAGATCGATATAAAAATGGG...,"P53, BCC7, LFS1, BMFS5, TRP53",SKV*SHRPGSR*LLGSGDTLRSGWERAFHDGDTLPWIG*APD*T**...,25,41,ATCGCTATCTGAGCAGCGCTCATGGTGGGGGCAGCGCCTCACAACC...,"530941076, 201744589, 138729528, 28934576, 115...",5,Yes,454545707
1,2771,GNAI2,G protein subunit alpha i2,3p21.31,NC_000003,"NC_000003.12 Homo sapiens chromosome 3, GRCh38...",568815595,50227068,50263357,plus,CATCACCGTCTAATCTCTGCTGTGAAGTGGAAGCGCGAGAAGGAGG...,"GIP, HG1C, GNAI2B, H_LUCA15.1, H_LUCA16.1",HHRLISAVKWKREKEGASHDGGCEDARLDEAERRVSLGTF*G*ADG...,7,47,TAGAGCCTGCAGGCAAGCATGTCACAGCCCACTGAGAGGGTCTTCA...,NaN,0,Yes,121023
2,5573,PRKAR1A,protein kinase cAMP-dependent type I regulator...,17q24.2,NC_000017,"NC_000017.11 Homo sapiens chromosome 17, GRCh3...",568815581,68413623,68551315,plus,AGCTGTGGTGGGCTCCACCCAGTTCGAGCTTCCCGGCTGCTTTGGT...,"CAR, CNC, CNC1, PKR1, TSE1, ADOHR, PPNAD1, PRK...",SCGGLHPVRASRLLWLPNQAWAMAGAPPPASLPPCSLISDCCASNQ...,11,107,GAGCTGTCGCCTAGCCGCTATCGCAGAGTGGAGCGGGGCTGGGAGC...,139620139,1,Yes,4506063


In [69]:
Entrez.email="dbsnp-user@nih.gov"

def has_3d_structure(geneid='7157'):
    try:
        # Найдём белок по названию гена
        query = f"({geneid}[UID] OR {geneid}[Gene ID]) AND Homo sapiens[Organism]"
        handle = Entrez.esearch(db="protein", term=query, retmax=5)
        record = Entrez.read(handle)
        handle.close()

        if not record['IdList']:
            return "No", None, None

        protein_id = record['IdList'][0]
        time.sleep(0.5)

        # Проверим, связан ли белок со структурой
        link_handle = Entrez.elink(dbfrom="protein", db="structure", id=protein_id)
        link_record = Entrez.read(link_handle)
        link_handle.close()

        # Есть ли запись в базе structure?
        if link_record[0].get("LinkSetDb"):
            time.sleep(0.5)
            # Получаем последовательность аминокислот
            handle_protein = Entrez.efetch(db="protein", id=protein_id, rettype='fasta', retmode='text')
            record = SeqIO.read(handle_protein, 'fasta' )
            handle_protein.close()
            protein_chain=str(record.seq)
            time.sleep(0.5)
            return "Yes", protein_id, protein_chain
        else:
            return "No", None, None

    except Exception as e:
        print("Ошибка:", e)
        return "Error", None, None

In [70]:
def check_3D_structure(db=genes_table):
  output=db
  output[['Has 3D structure', 'Protein id with 3D', 'Structure protein 3D']]=None
  for idx, row in output.iterrows():
      output.loc[idx, 'Has 3D structure'], output.loc[idx, 'Protein id with 3D'], output.loc[idx, 'Structure protein 3D']=has_3d_structure(row['geneid'])
  return output

In [71]:
genes_table=check_3D_structure(genes_table)
display(genes_table)

,geneid,gene_symbol,gene_desc,locus,chromosome,chromosome_desc,seq_id,seq_start,seq_end,strand,...,gene_syn,protein,q_transcripts,len_long_prot,seq_dna,snps_id,q_snps,Has 3D structure,Protein id with 3D,Structure protein 3D
0,7157,TP53,tumor protein p53,17p13.1,NC_000017,"NC_000017.11 Homo sapiens chromosome 17, GRCh3...",568815581,7668421,7687489,minus,...,"P53, BCC7, LFS1, BMFS5, TRP53",SKV*SHRPGSR*LLGSGDTLRSGWERAFHDGDTLPWIG*APD*T**...,25,41,ATCGCTATCTGAGCAGCGCTCATGGTGGGGGCAGCGCCTCACAACC...,"530941076, 201744589, 138729528, 28934576, 115...",5,Yes,454545707,MAIYKQSQHMTEVVRRCPHHERCSDSDGLAPPQHLIRVEGNLRVEY...
1,2771,GNAI2,G protein subunit alpha i2,3p21.31,NC_000003,"NC_000003.12 Homo sapiens chromosome 3, GRCh38...",568815595,50227068,50263357,plus,...,"GIP, HG1C, GNAI2B, H_LUCA15.1, H_LUCA16.1",HHRLISAVKWKREKEGASHDGGCEDARLDEAERRVSLGTF*G*ADG...,7,47,TAGAGCCTGCAGGCAAGCATGTCACAGCCCACTGAGAGGGTCTTCA...,NaN,0,Yes,121023,MGCTVSAEDKAAAERSKMIDKNLREDGEKAAREVKLLLLGAGESGK...
2,5573,PRKAR1A,protein kinase cAMP-dependent type I regulator...,17q24.2,NC_000017,"NC_000017.11 Homo sapiens chromosome 17, GRCh3...",568815581,68413623,68551315,plus,...,"CAR, CNC, CNC1, PKR1, TSE1, ADOHR, PPNAD1, PRK...",SCGGLHPVRASRLLWLPNQAWAMAGAPPPASLPPCSLISDCCASNQ...,11,107,GAGCTGTCGCCTAGCCGCTATCGCAGAGTGGAGCGGGGCTGGGAGC...,139620139,1,Yes,4506063,MESGSTAASEEARSLRECELYVQKHNIQALLKDSIVQLCTARPERP...


In [72]:
genes_table.to_csv('genes_table.csv', index=False)

In [77]:
genes_table= pd.read_csv('genes_table.csv')

In [78]:
genes_table.head()

,geneid,gene_symbol,gene_desc,locus,chromosome,chromosome_desc,seq_id,seq_start,seq_end,strand,...,gene_syn,protein,q_transcripts,len_long_prot,seq_dna,snps_id,q_snps,Has 3D structure,Protein id with 3D,Structure protein 3D
0,7157,TP53,tumor protein p53,17p13.1,NC_000017,"NC_000017.11 Homo sapiens chromosome 17, GRCh3...",568815581,7668421,7687489,minus,...,"P53, BCC7, LFS1, BMFS5, TRP53",SKV*SHRPGSR*LLGSGDTLRSGWERAFHDGDTLPWIG*APD*T**...,25,41,ATCGCTATCTGAGCAGCGCTCATGGTGGGGGCAGCGCCTCACAACC...,"530941076, 201744589, 138729528, 28934576, 115...",5,Yes,454545707,MAIYKQSQHMTEVVRRCPHHERCSDSDGLAPPQHLIRVEGNLRVEY...
1,2771,GNAI2,G protein subunit alpha i2,3p21.31,NC_000003,"NC_000003.12 Homo sapiens chromosome 3, GRCh38...",568815595,50227068,50263357,plus,...,"GIP, HG1C, GNAI2B, H_LUCA15.1, H_LUCA16.1",HHRLISAVKWKREKEGASHDGGCEDARLDEAERRVSLGTF*G*ADG...,7,47,TAGAGCCTGCAGGCAAGCATGTCACAGCCCACTGAGAGGGTCTTCA...,NaN,0,Yes,121023,MGCTVSAEDKAAAERSKMIDKNLREDGEKAAREVKLLLLGAGESGK...
2,5573,PRKAR1A,protein kinase cAMP-dependent type I regulator...,17q24.2,NC_000017,"NC_000017.11 Homo sapiens chromosome 17, GRCh3...",568815581,68413623,68551315,plus,...,"CAR, CNC, CNC1, PKR1, TSE1, ADOHR, PPNAD1, PRK...",SCGGLHPVRASRLLWLPNQAWAMAGAPPPASLPPCSLISDCCASNQ...,11,107,GAGCTGTCGCCTAGCCGCTATCGCAGAGTGGAGCGGGGCTGGGAGC...,139620139,1,Yes,4506063,MESGSTAASEEARSLRECELYVQKHNIQALLKDSIVQLCTARPERP...


# ЭТУ ПОСЛЕДОВАТЕЛЬНОСТЬ АМИНОКИСЛОТ ЗАГРУЗИМ В AlphaFold ДЛЯ 3D МОДЕЛИРОВАНИЯ

In [84]:
print(genes_table['Structure protein 3D'][0])

MAIYKQSQHMTEVVRRCPHHERCSDSDGLAPPQHLIRVEGNLRVEYLDDRNTFRHSVVVPYEPPEVGSDCTTIHYNYMCNSSCMGGMNRRPILTIITLEDSSGNLLGRNSFEVRVCACPGRDRRTEEENLRKKGEPHHELPPGSTKRALPNNTSSSPQPKKKPLDGEYFTLQMLLDLRWCYFLINSS
